<a href="https://colab.research.google.com/github/EmmaHsueh/PL_project/blob/main/Ebook_Reading_Notes_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# E-book Reading Notes System

[google sheet link path](https://docs.google.com/spreadsheets/d/1xmh_N_DYzs0MwSc0qd1e1LAvulSNYyXUULJRslUacuw/edit?usp=sharing)

In [ ]:
!pip install -q PyMuPDF pandas openpyxl gradio google-generativeai gspread pillow pytz


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 30.4 MB/s eta 0:00:00


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
import fitz  # PyMuPDF
import pandas as pd
from datetime import datetime, timedelta
import pytz
import gradio as gr
import json
import re
from pathlib import Path
import google.generativeai as genai
import os
from collections import defaultdict

# 設定台灣時區
TIMEZONE = "Asia/Taipei"
TW_TZ = pytz.timezone(TIMEZONE)

# Google Sheets 設定
SHEET_URL = "https://docs.google.com/spreadsheets/d/1xmh_N_DYzs0MwSc0qd1e1LAvulSNYyXUULJRslUacuw/edit?usp=sharing"


In [ ]:
class EbookNotebook:
    def __init__(self, gemini_api_key=None):
        """初始化電子書筆記本"""
        self.api_key = gemini_api_key
        if gemini_api_key:
            genai.configure(api_key=gemini_api_key)
            self.model = genai.GenerativeModel('gemini-2.0-flash-exp')
        else:
            self.model = None

        # 初始化 Google Sheets
        try:
            creds, _ = default()
            self.gc = gspread.authorize(creds)
            self.spreadsheet = self.gc.open_by_url(SHEET_URL)
            print("✅ Google Sheets 連接成功！")
        except Exception as e:
            print(f"⚠️ Google Sheets 連接失敗：{e}")
            self.gc = None
            self.spreadsheet = None

        self.reading_log = []
        self.highlights = []
        self.notes = []

    def init_sheets(self):
        """初始化 Google Sheets 工作表結構"""
        if not self.spreadsheet:
            return "❌ Google Sheets 未連接"

        try:
            # 檢查並建立需要的工作表
            sheet_names = [ws.title for ws in self.spreadsheet.worksheets()]

            # 工作表1：書籍資訊
            if "書籍資訊" not in sheet_names:
                ws = self.spreadsheet.add_worksheet(title="書籍資訊", rows=100, cols=10)
                ws.update('A1:E1', [["書名", "作者", "總頁數", "上傳時間", "檔案名稱"]])

            # 工作表2：閱讀記錄
            if "閱讀記錄" not in sheet_names:
                ws = self.spreadsheet.add_worksheet(title="閱讀記錄", rows=1000, cols=10)
                ws.update('A1:F1', [["日期", "書名", "已讀頁數", "目標頁數", "完成率", "狀態"]])

            # 工作表3：螢光筆標記
            if "螢光筆標記" not in sheet_names:
                ws = self.spreadsheet.add_worksheet(title="螢光筆標記", rows=1000, cols=10)
                ws.update('A1:D1', [["書名", "頁碼", "標記內容", "記錄時間"]])

            # 工作表4：本週金句
            if "本週金句" not in sheet_names:
                ws = self.spreadsheet.add_worksheet(title="本週金句", rows=500, cols=10)
                ws.update('A1:C1', [["週次", "金句", "來源書籍"]])

            # 工作表5：AI摘要
            if "AI摘要" not in sheet_names:
                ws = self.spreadsheet.add_worksheet(title="AI摘要", rows=500, cols=10)
                ws.update('A1:D1', [["書名", "頁碼/章節", "摘要內容", "生成時間"]])

            return "✅ Google Sheets 工作表初始化完成！"

        except Exception as e:
            return f"❌ 初始化失敗：{str(e)}"

    def save_book_info_to_sheet(self, book_info):
        """儲存書籍資訊到 Google Sheets"""
        if not self.spreadsheet:
            return False

        try:
            ws = self.spreadsheet.worksheet("書籍資訊")
            now = datetime.now(TW_TZ).strftime('%Y-%m-%d %H:%M:%S')

            row = [
                book_info.get("title", "未知"),
                book_info.get("author", "未知"),
                book_info.get("total_pages", 0),
                now,
                book_info.get("filename", "")
            ]

            ws.append_row(row)
            return True
        except Exception as e:
            print(f"儲存書籍資訊失敗：{e}")
            return False

    def save_reading_record_to_sheet(self, book_title, pages_read, target=50):
        """儲存閱讀記錄到 Google Sheets"""
        if not self.spreadsheet:
            return False

        try:
            ws = self.spreadsheet.worksheet("閱讀記錄")
            now = datetime.now(TW_TZ)
            completion_rate = (pages_read / target) * 100
            status = "達標" if pages_read >= target else "未達標"

            row = [
                now.strftime('%Y-%m-%d'),
                book_title,
                pages_read,
                target,
                f"{completion_rate:.1f}%",
                status
            ]

            ws.append_row(row)
            return True
        except Exception as e:
            print(f"儲存閱讀記錄失敗：{e}")
            return False

    def save_highlights_to_sheet(self, book_title, highlights):
        """儲存螢光筆標記到 Google Sheets"""
        if not self.spreadsheet or not highlights:
            return False

        try:
            ws = self.spreadsheet.worksheet("螢光筆標記")
            now = datetime.now(TW_TZ).strftime('%Y-%m-%d %H:%M:%S')

            rows = []
            for h in highlights:
                rows.append([
                    book_title,
                    h.get("page", ""),
                    h.get("text", ""),
                    now
                ])

            if rows:
                ws.append_rows(rows)
            return True
        except Exception as e:
            print(f"儲存標記失敗：{e}")
            return False

    def save_quotes_to_sheet(self, book_title, quotes):
        """儲存金句到 Google Sheets"""
        if not self.spreadsheet or not quotes:
            return False

        try:
            ws = self.spreadsheet.worksheet("本週金句")
            now = datetime.now(TW_TZ)
            week_num = now.isocalendar()[1]  # 取得週數

            rows = []
            for quote in quotes:
                rows.append([
                    f"{now.year}-W{week_num:02d}",
                    quote,
                    book_title
                ])

            if rows:
                ws.append_rows(rows)
            return True
        except Exception as e:
            print(f"儲存金句失敗：{e}")
            return False

    def save_summary_to_sheet(self, book_title, chapter_info, summary):
        """儲存 AI 摘要到 Google Sheets"""
        if not self.spreadsheet:
            return False

        try:
            ws = self.spreadsheet.worksheet("AI摘要")
            now = datetime.now(TW_TZ).strftime('%Y-%m-%d %H:%M:%S')

            row = [
                book_title,
                chapter_info,
                summary,
                now
            ]

            ws.append_row(row)
            return True
        except Exception as e:
            print(f"儲存摘要失敗：{e}")
            return False

    def read_from_sheet(self, sheet_name):
        """從 Google Sheets 讀取資料"""
        if not self.spreadsheet:
            return None

        try:
            ws = self.spreadsheet.worksheet(sheet_name)
            data = ws.get_all_records()
            return pd.DataFrame(data)
        except Exception as e:
            print(f"讀取 {sheet_name} 失敗：{e}")
            return None

    def extract_pdf_content(self, pdf_path):
        """提取 PDF 內容與螢光筆標記"""
        try:
            doc = fitz.open(pdf_path)
            content = {
                "total_pages": len(doc),
                "text_by_page": {},
                "highlights": [],
                "metadata": doc.metadata,
                "filename": Path(pdf_path).name
            }

            for page_num in range(len(doc)):
                page = doc[page_num]

                # 提取文字
                text = page.get_text()
                content["text_by_page"][page_num + 1] = text

                # 提取螢光筆標記（黃色標記）
                annotations = page.annots()
                if annotations:
                    for annot in annotations:
                        if annot.type[0] == 8:  # 8 = 螢光筆標記
                            rect = annot.rect
                            marked_text = page.get_text("text", clip=rect)
                            if marked_text.strip():
                                content["highlights"].append({
                                    "page": page_num + 1,
                                    "text": marked_text.strip()
                                })

            doc.close()
            return content

        except Exception as e:
            return {"error": str(e)}

    def detect_chapters(self, content):
        """偵測章節"""
        chapters = []
        for page_num, text in content["text_by_page"].items():
            lines = text.split('\n')
            for i, line in enumerate(lines):
                if re.match(r'^(第[一二三四五六七八九十\d]+章|Chapter\s+\d+|CHAPTER\s+\d+)', line.strip()):
                    chapters.append({
                        "title": line.strip(),
                        "page": page_num
                    })
        return chapters

    def calculate_weekly_stats(self, start_date, end_date, pages_read):
        """計算本週閱讀統計"""
        now = datetime.now(TW_TZ)
        week_start = now - timedelta(days=now.weekday())
        week_end = week_start + timedelta(days=6)

        stats = {
            "week_range": f"{week_start.strftime('%Y-%m-%d')} ~ {week_end.strftime('%Y-%m-%d')}",
            "pages_read": pages_read,
            "target": 20,
            "completion_rate": (pages_read / 20) * 100,
            "status": "達標" if pages_read >= 20 else "未達標"
        }
        return stats

    def generate_chapter_summary(self, chapter_text, chapter_title=""):
        """使用 Gemini AI 生成章節摘要"""
        if not self.model:
            return "請先設定 Gemini API Key 以使用 AI 功能"

        try:
            prompt = f"""請為以下章節內容生成簡潔的摘要（約100-150字）：

章節標題：{chapter_title}

內容：
{chapter_text[:3000]}

請用繁體中文回答，重點說明：
1. 本章主要論點
2. 關鍵概念
3. 重要結論"""

            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"AI 摘要生成失敗：{str(e)}"

    def extract_weekly_quotes(self, highlights):
        """使用 Gemini AI 從標記中提取本週金句"""
        if not self.model or not highlights:
            return []

        try:
            highlights_text = "\n\n".join([f"[第{h['page']}頁] {h['text']}" for h in highlights[:20]])

            prompt = f"""從以下閱讀標記中，挑選出 3-5 句最有價值的金句：

{highlights_text}

請以 JSON 格式回傳，格式如下：
{{"quotes": ["金句1", "金句2", "金句3"]}}

只回傳 JSON，不要其他文字。"""

            response = self.model.generate_content(prompt)
            result_text = response.text.strip()

            # 清理 markdown 程式碼區塊標記
            if result_text.startswith("```json"):
                result_text = result_text[7:]
            if result_text.startswith("```"):
                result_text = result_text[3:]
            if result_text.endswith("```"):
                result_text = result_text[:-3]

            result = json.loads(result_text.strip())
            return result.get("quotes", [])
        except Exception as e:
            print(f"金句提取失敗：{e}")
            return []

    def answer_question(self, question, content):
        """智能問答功能"""
        if not self.model:
            return "請先設定 Gemini API Key 以使用 AI 功能"

        try:
            all_text = "\n".join([text[:1000] for text in list(content["text_by_page"].values())[:10]])

            prompt = f"""根據以下書籍內容回答問題：

書籍內容摘錄：
{all_text}

問題：{question}

請用繁體中文回答，並引用相關段落。"""

            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"問答失敗：{str(e)}"

    def generate_yearly_report(self):
        """從 Google Sheets 生成年度閱讀報告"""
        if not self.spreadsheet:
            return "❌ Google Sheets 未連接"

        try:
            # 讀取閱讀記錄
            df = self.read_from_sheet("閱讀記錄")
            if df is None or df.empty:
                return "📊 目前尚無閱讀記錄"

            now = datetime.now(TW_TZ)
            year = now.year

            # 篩選今年的記錄
            df['日期'] = pd.to_datetime(df['日期'])
            df_year = df[df['日期'].dt.year == year]

            if df_year.empty:
                return f"📊 {year} 年尚無閱讀記錄"

            # 計算統計
            total_books = df_year['書名'].nunique()
            total_pages = df_year['已讀頁數'].sum()
            avg_pages = total_pages / len(df_year) if len(df_year) > 0 else 0

            # 計算達標率
            achieved = len(df_year[df_year['狀態'] == '達標'])
            achievement_rate = (achieved / len(df_year)) * 100 if len(df_year) > 0 else 0

            report = f"""
📚 {year} 年度閱讀報告

📊 閱讀統計：
- 閱讀書籍：{total_books} 本
- 總閱讀頁數：{total_pages} 頁
- 記錄次數：{len(df_year)} 次
- 平均每次：{avg_pages:.1f} 頁

🎯 目標達成：
- 達標次數：{achieved} / {len(df_year)}
- 達標率：{achievement_rate:.1f}%
- 週平均頁數：{total_pages / 52:.1f} 頁

📖 最常閱讀：
{df_year['書名'].value_counts().head(3).to_string()}

生成時間：{now.strftime('%Y-%m-%d %H:%M:%S')}
"""
            return report

        except Exception as e:
            return f"❌ 生成報告失敗：{str(e)}"


In [ ]:
# ===== 第四步：Gradio 介面 =====

def create_gradio_interface():
    """建立 Gradio 使用者介面"""

    notebook = None
    current_content = None
    current_book_title = None

    def initialize_notebook(api_key):
        """初始化筆記本"""
        nonlocal notebook
        if not api_key:
            return "⚠️ 請輸入 Gemini API Key"
        notebook = EbookNotebook(api_key)
        init_msg = notebook.init_sheets()
        return f"✅ 系統初始化成功！\n{init_msg}\n\n請上傳 PDF 檔案"

    def process_pdf(pdf_file, pages_read):
        """處理 PDF 檔案"""
        nonlocal current_content, current_book_title
        if not notebook:
            return "❌ 請先初始化系統", None, None, None

        if pdf_file is None:
            return "❌ 請上傳 PDF 檔案", None, None, None

        # 提取內容
        current_content = notebook.extract_pdf_content(pdf_file.name)

        if "error" in current_content:
            return f"❌ PDF 解析失敗：{current_content['error']}", None, None, None

        # 取得書名
        current_book_title = current_content['metadata'].get('title', '未知書籍')
        if current_book_title == '未知書籍':
            current_book_title = current_content['filename']

        # 儲存書籍資訊到 Google Sheets
        book_info = {
            "title": current_book_title,
            "author": current_content['metadata'].get('author', '未知'),
            "total_pages": current_content['total_pages'],
            "filename": current_content['filename']
        }
        notebook.save_book_info_to_sheet(book_info)

        # 計算統計
        stats = notebook.calculate_weekly_stats(None, None, pages_read)

        # 儲存閱讀記錄到 Google Sheets
        notebook.save_reading_record_to_sheet(current_book_title, pages_read)

        # 儲存螢光筆標記到 Google Sheets
        notebook.save_highlights_to_sheet(current_book_title, current_content["highlights"])

        # 提取金句
        quotes = notebook.extract_weekly_quotes(current_content["highlights"])

        # 儲存金句到 Google Sheets
        notebook.save_quotes_to_sheet(current_book_title, quotes)

        # 生成結果文字
        result = f"""
✅ PDF 分析完成！資料已同步至 Google Sheets

📖 書籍資訊：
- 書名：{current_book_title}
- 總頁數：{current_content['total_pages']}
- 發現標記：{len(current_content['highlights'])} 處

📊 本週閱讀統計：
- 週期：{stats['week_range']}
- 已讀頁數：{stats['pages_read']} / {stats['target']}
- 完成率：{stats['completion_rate']:.1f}%
- 狀態：{stats['status']}

💾 已儲存至工作表：
- ✅ 書籍資訊
- ✅ 閱讀記錄
- ✅ 螢光筆標記（{len(current_content['highlights'])} 筆）
- ✅ 本週金句（{len(quotes)} 句）
"""

        # 準備標記列表
        highlights_text = "\n\n".join([
            f"📍 第 {h['page']} 頁：\n{h['text']}"
            for h in current_content["highlights"][:10]
        ])

        if len(current_content["highlights"]) > 10:
            highlights_text += f"\n\n... 還有 {len(current_content['highlights']) - 10} 筆標記"

        # 準備金句列表
        quotes_text = "\n\n".join([f"💎 {q}" for q in quotes]) if quotes else "尚未提取金句"

        return result, highlights_text, quotes_text, current_content

    def generate_summary(chapter_num):
        """生成章節摘要"""
        if not current_content:
            return "❌ 請先上傳並分析 PDF"

        try:
            chapter_num = int(chapter_num)
            if chapter_num < 1 or chapter_num > current_content["total_pages"]:
                return f"❌ 頁碼超出範圍（1-{current_content['total_pages']}）"

            chapter_text = current_content["text_by_page"].get(chapter_num, "")
            if not chapter_text.strip():
                return f"❌ 第 {chapter_num} 頁無內容"

            summary = notebook.generate_chapter_summary(chapter_text, f"第 {chapter_num} 頁")

            # 儲存摘要到 Google Sheets
            notebook.save_summary_to_sheet(current_book_title, f"第 {chapter_num} 頁", summary)

            return f"📝 第 {chapter_num} 頁摘要：\n\n{summary}\n\n✅ 已儲存至 Google Sheets「AI摘要」工作表"
        except ValueError:
            return "❌ 請輸入有效的頁碼數字"

    def ask_question(question):
        """智能問答"""
        if not current_content:
            return "❌ 請先上傳並分析 PDF"

        if not question.strip():
            return "❌ 請輸入問題"

        answer = notebook.answer_question(question, current_content)
        return f"🤔 問題：{question}\n\n💡 回答：\n{answer}"

    def view_sheet_data(sheet_name):
        """查看 Google Sheets 資料"""
        if not notebook:
            return "❌ 請先初始化系統"

        df = notebook.read_from_sheet(sheet_name)
        if df is None:
            return f"❌ 無法讀取「{sheet_name}」工作表"

        if df.empty:
            return f"📊 「{sheet_name}」工作表目前無資料"

        return df

    def generate_year_report():
        """生成年度報告"""
        if not notebook:
            return "❌ 請先初始化系統"

        report = notebook.generate_yearly_report()
        return report

    # 建立介面
    with gr.Blocks(title="📚 電子書閱讀筆記本", theme=gr.themes.Soft()) as app:
        gr.Markdown("# 📚 電子書閱讀筆記本")
        gr.Markdown("上傳 PDF 電子書，系統將自動提取螢光筆標記、生成摘要與金句，並同步至 Google Sheets！")

        with gr.Tab("🔧 系統設定"):
            gr.Markdown(f"### Google Sheets 連結\n{SHEET_URL}")
            api_key_input = gr.Textbox(
                label="Gemini API Key",
                placeholder="輸入你的 Gemini API Key...",
                type="password"
            )

            init_btn = gr.Button("初始化系統", variant="primary")
            init_output = gr.Textbox(label="初始化狀態", lines=5)

            init_btn.click(initialize_notebook, inputs=[api_key_input], outputs=[init_output])

        with gr.Tab("📖 上傳分析"):
            with gr.Row():
                with gr.Column():
                    pdf_input = gr.File(label="上傳 PDF 電子書", file_types=[".pdf"])
                    pages_input = gr.Number(label="本週已讀頁數", value=30, minimum=0)
                    analyze_btn = gr.Button("開始分析", variant="primary")

                with gr.Column():
                    analysis_output = gr.Textbox(label="分析結果", lines=20)

            with gr.Row():
                highlights_output = gr.Textbox(label="📍 螢光筆標記", lines=10)
                quotes_output = gr.Textbox(label="💎 本週金句", lines=10)

            analyze_btn.click(
                process_pdf,
                inputs=[pdf_input, pages_input],
                outputs=[analysis_output, highlights_output, quotes_output, gr.State()]
            )

        with gr.Tab("📝 AI 功能"):
            with gr.Row():
                with gr.Column():
                    gr.Markdown("### 章節摘要生成")
                    chapter_input = gr.Number(label="輸入頁碼", value=1, minimum=1)
                    summary_btn = gr.Button("生成摘要", variant="primary")
                    summary_output = gr.Textbox(label="摘要結果", lines=10)

                with gr.Column():
                    gr.Markdown("### 智能問答")
                    question_input = gr.Textbox(
                        label="輸入問題",
                        placeholder="例如：書中關於時間管理的建議是什麼？"
                    )
                    qa_btn = gr.Button("提問", variant="primary")
                    qa_output = gr.Textbox(label="回答", lines=10)

            summary_btn.click(generate_summary, inputs=[chapter_input], outputs=[summary_output])
            qa_btn.click(ask_question, inputs=[question_input], outputs=[qa_output])

        with gr.Tab("📊 查看資料"):
            gr.Markdown("### 從 Google Sheets 查看已儲存的資料")

            with gr.Row():
                sheet_selector = gr.Dropdown(
                    choices=["書籍資訊", "閱讀記錄", "螢光筆標記", "本週金句", "AI摘要"],
                    label="選擇工作表",
                    value="閱讀記錄"
                )
                view_btn = gr.Button("查看資料", variant="primary")

            sheet_data_output = gr.Dataframe(label="資料內容")

            view_btn.click(view_sheet_data, inputs=[sheet_selector], outputs=[sheet_data_output])

        with gr.Tab("📈 年度報告"):
            gr.Markdown("### 從 Google Sheets 生成年度閱讀報告")

            year_report_btn = gr.Button("生成年度報告", variant="primary", size="lg")
            year_report_output = gr.Textbox(label="年度報告", lines=20)

            year_report_btn.click(generate_year_report, outputs=[year_report_output])

        gr.Markdown("""
---
### 💡 使用說明

#### 1️⃣ 系統設定
- 輸入 Gemini API Key（從 [Google AI Studio](https://aistudio.google.com/app/apikey) 取得）
- 點擊「初始化系統」建立 Google Sheets 工作表結構

#### 2️⃣ 上傳分析
- 上傳有螢光筆標記的 PDF
- 設定本週閱讀頁數
- 系統會自動提取標記、生成金句並同步至 Google Sheets

#### 3️⃣ AI 功能
- **章節摘要**：輸入頁碼，AI 自動生成該頁摘要
- **智能問答**：詢問書籍內容相關問題

#### 4️⃣ 查看資料
- 直接從 Google Sheets 讀取並顯示已儲存的資料
- 支援查看 5 個工作表的內容

#### 5️⃣ 年度報告
- 基於 Google Sheets 的閱讀記錄生成年度統計報告

---

### 📊 Google Sheets 工作表說明

| 工作表 | 內容 |
|--------|------|
| 📚 **書籍資訊** | 記錄所有上傳的書籍（書名、作者、頁數等） |
| 📊 **閱讀記錄** | 每次閱讀的進度與達標狀況 |
| 📍 **螢光筆標記** | 所有標記的內容（含頁碼） |
| 💎 **本週金句** | AI 精選的金句 |
| 📝 **AI摘要** | 每個頁面/章節的 AI 摘要 |



        """)

    return app


In [ ]:
if __name__ == "__main__":
    print("🚀 啟動電子書閱讀筆記本系統...")
    print(f"📊 Google Sheets 連結：{SHEET_URL}")
    print("⚙️  正在建立介面...")

    # 建立並啟動介面
    app = create_gradio_interface()

    print("✅ 介面建立成功！")
    print("🌐 正在啟動 Gradio 伺服器...")

    # 啟動 Gradio（share=True 可以產生公開連結）
    app.launch(
        share=True,      # 產生公開分享連結
        debug=True,      # 開啟除錯模式
        server_name="0.0.0.0",  # 允許外部訪問
        server_port=7860        # 指定端口
    )

    print("🎉 系統啟動完成！請點擊上方連結開始使用")

🚀 啟動電子書閱讀筆記本系統...
📊 Google Sheets 連結：https://docs.google.com/spreadsheets/d/1xmh_N_DYzs0MwSc0qd1e1LAvulSNYyXUULJRslUacuw/edit?usp=sharing
⚙️  正在建立介面...
✅ 介面建立成功！
🌐 正在啟動 Gradio 伺服器...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://56ee03c27aeac4242b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


✅ Google Sheets 連接成功！


✅ Google Sheets 連接成功！


✅ Google Sheets 連接成功！
